In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("Spotify Million Song Dataset_exported.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [15]:
df = df.drop(['link'], axis=1)
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   text    57650 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [16]:
indent = ["\n", "\r", "\t"]
def remove_indents(text, indent):
    for elements in indent:
        text = text.replace(elements, '')
    return text

df['text'] = df['text'].apply(lambda elements: remove_indents(elements, indent))

In [17]:
df['text'] = df['text'].str.lower()


In [18]:
import re
def remove_words_with_apostrophes(input_string):
    pattern = r'\b\w+\'\w+\b'
    result = re.sub(pattern, '', input_string)
    return result

df['text'] = df['text'].apply(lambda elements: remove_words_with_apostrophes(elements))

In [19]:
import string
def remove_punctuation(input_string):
    translation_table = str.maketrans('', '', string.punctuation)
    result = input_string.translate(translation_table)
    return result

df['text'] = df['text'].apply(lambda elements: remove_punctuation(elements))

In [20]:

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lyrics = df['text'].apply(lambda x: ' '.join([ps.stem(word) for word in re.findall(r'\b\w+\b', x.lower()) if word not in stop_words]))

In [21]:
# Vectorization using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(lyrics)

In [29]:
df['text'][15]

'i can hear how you work practising hard  playing night and day wooh  and it sounds better now yes you improve  everytime you play wooh  but  bad   so sad  and  only smiling  when you play your violin    dumb dumb diddle to be your fiddle  to be so near you and not just hear you  dumb dumb diddle to be your fiddle  i think then maybe  see me baby   be mine  and  be together all the time  wish i was dumb dumb diddle your darling fiddle    but i think you  know that i exist   the quiet kind wooh  from the day when i first listened to you   been on my mind wooh  you  care   not fair  and  only smiling  when you play your violin    dumb dumb diddle to be your fiddle  to be so near you and not just hear you  dumb dumb diddle to be your fiddle  i think then maybe  see me baby   be mine  and  be together all the time    dumb dumb diddle to be your fiddle  to be so near you and not just hear you  dumb dumb diddle to be your fiddle  i think then maybe  see me baby   be mine  and  be together al

In [48]:
# Input snippet
input_snippet = df['text'][15]
input_snippet = "i can hear how you work practising hard  playing night and day wooh  and it sounds better now yes you improve  everytime you play wooh  but  bad   so sad  and  only smiling  when you play your violin    dumb dumb diddle to be your fiddle  to be so near you and not just hear you  dumb dumb diddle to be your fiddle  i think then maybe  see me baby   be mine  and  be together all the time  wish i was dumb dumb diddle your darling fiddle    but i think you  know that i exist   the quiet kind wooh  "

In [49]:
input_snippet=remove_words_with_apostrophes(input_snippet)
input_snippet=remove_punctuation(input_snippet)

In [51]:
# Preprocess input snippet

input_snippet = ' '.join([ps.stem(word) for word in re.findall(r'\b\w+\b', input_snippet.lower()) if word not in stop_words])

In [52]:
# Vectorize input snippet
input_snippet_vector = vectorizer.transform([input_snippet])

In [53]:
# Calculate similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(X, input_snippet_vector)

In [54]:
# Identify the unique song
unique_song_index = similarity_scores.argmax()
unique_song = df.iloc[unique_song_index]

In [55]:
print(f'Artist: {unique_song["artist"]}')
print(f'Song: {unique_song["song"]}')

Artist: ABBA
Song: Dum Dum Diddle


In [56]:
print(df['artist'][15])
print(df['song'][15])

ABBA
Dum Dum Diddle
